In [1]:
import sys
sys.path.append('../codes')

# 7. 함수 데커레이터와 클로저

__함수 데커레이터__ 는 소스 코드에 있는 함수를 '표시'해서 함수의 작동을 개선할 수 있게 해줍니다. 

데커레이터를 정확히 알기 위해 다음과 같은 내용을 먼저 알아야 합니다.

1. 파이썬이 데커레이터 구문을 평가하는 방식
2. 변수가 지역 변수인지 파이썬이 판단하는 방식
3. 클로저의 존재 이유와 작동 방식
4. nonlocal로 해결할 수 있는 문제

이 후에는 다음 주제를 심도 있게 다룰 수 있습니다.

1. 잘 작동하는 데커레이터 구현하기
2. 표준 라이브러리에서 제공하는 데커레이터
3. 매개변수화된 데커레이터 구현하기

## 7.1 데커레이터 기본 지식

데커레이터는 다른 함수를 인수로 받는 콜러블(데커레이트된 함수)입니다.

데커레이터는 데커레이트된 함수에 어떤 처리를 수행하고, 함수를 반환하거나 함수를 다른 함수나 콜러블의 객체로 대체합니다. 

```
#1
@decorate
def target():
    print('running target()')
```

```
#2
def target():
    print('running target()')  
target = decorate(target)
```

`target()`은 `decorate(target)`이 반환된 함수를 가리키게 됩니다.

In [2]:
## ex 7.1
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')
    
target()

running inner()


In [3]:
target

<function __main__.deco.<locals>.inner()>

데커레이터는 __편리 구문(syntatic sugar)__ 일 뿐입니다. 

> __syntatic sugar__
-  프로그래밍 언어차원에서 제공되는 논리적으로 간결하게 표현하는 것
- 중복되는 로직을 간결하게 표현하기 위함

런타임에 프로그램 행위를 변경하는 __메타프로그래밍__ 을 할 때 데커레이터가 상당히 편리합니다.

>__런타임__
    - 프로그램이 실행되고 있는 동안의 동작
__메타프로그래밍__
    - 자기 자신 혹은 다른 컴퓨터 프로그램을 데이터로 취급하여 프로그램을 작성-수정

1. __데커레이터는 데커레이트된 함수를 다른 함수로 대체__


2. __데커레이터는 모듈이 로딩될 때 바로 실행__

## 7.2 파이썬이 데커레이터를 실행하는 시점

데커레이터는 데커레이트된 함수가 정의된 직후, 즉 모듈을 로딩하는 시점(__임포트 타임__)에 실행됩니다.

In [4]:
## ex 7.2
registry = []
def register(func):
    print(f'running register{func}')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
def main():
    print('running main()')
    print('register -> ', registry)
    f1()
    f2()
    f3()

running register<function f1 at 0x10c57c9e0>
running register<function f2 at 0x10c57ccb0>


`register()`가 호출될 때 데커레이트 된 함수를 인수로 받으며 먼저 실행됩니다.

In [5]:
registry

[<function __main__.f1()>, <function __main__.f2()>]

In [6]:
f1()

running f1()


In [7]:
registry

[<function __main__.f1()>, <function __main__.f2()>]

In [8]:
main()

running main()
register ->  [<function f1 at 0x10c57c9e0>, <function f2 at 0x10c57ccb0>]
running f1()
running f2()
running f3()


모듈이 로딩된 후 `registery`는 데커레이트된 두 개의 함수(`f1`, `f2`)에 대한 참조를 가집니다.

In [9]:
import registration

running register<function f1 at 0x10c5aa440>
running register<function f2 at 0x10c5aa200>


In [10]:
registration.registry

[<function registration.f1()>, <function registration.f2()>]

데커레이터는 모듈이 임포트되자마자 실행되지만, 데커레이트된 함수는 명시적으로 호출될 때만 실행됨을 알 수 있습니다. 

이는 __임포트 타임__ 과 __런타임__ 의 차이를 명확히 보여줍니다.

1. 위 코드는 데커레이터 함수가 데커레이트되는 함수와 같은 모듈에 정의되어 있지만 일반적으로는 __데커레이터를 정의하는 모듈과 데커레이터를 적용하는 모듈을 분리해서 구현합니다.__


2. `register()` 데커레이터가 인수로 전달된 함수와 동일한 함수를 반환하지만, 일반적으로는 __데커레이터 내부 함수를 정의해서 반환합니다.__

## 7.3 데커레이터로 개선한 전략 패턴

In [11]:
## ex 7.3

promos = []
def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    '''충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용'''
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    '''20개 이상의 동일 상품을 구입하면 10% 할인 적용'''
    discount = 0
    for item in order.cart:
        if item.qunatity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    '''10종류 이상의 상품을 구입하면 전체 7% 할인 적용'''
    discount_items = {item.product for item in order.cart}
    if len(discount_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    '''최대로 할인받을 금액을 반환한다.'''
    return max(promo(order) for promo in promos)

1. 프로모션 전략 함수명이 특별한 형태로 되어 있을 필요 없습니다. 
2. `@promotion` 데커레이터는 데커레이트된 함수의 목적을 명확히 알려주며, 임시로 어떤 프로모션을 배제할 수 있습니다. 단지 데커레이터만 주석처리하면 됩니다.
3. 프로모션 할인 전략을 구현한 함수는 `@promotion` 데커레이터가 적용되는 한 어느 모듈에서든 정의할 수 있습니다. 

데코레이트된 함수가 아닌 내부 함수를 사용하는 코드는 제대로 작동하기 위해 거의 항상 __클로저__에 의존합니다. 

## 7.4 변수 범위 규칙

In [12]:
## ex 7.4
def f1(a):
    print(a)
    print(b)
    
f1(3)

3


NameError: name 'b' is not defined

In [13]:
b = 6
f1(3)

3
6


In [14]:
## ex 7.5
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

파이썬이 함수 본체를 컴파일할 때 `b`가 함수 안에서 할당되므로 `b`를 지역 변수로 판단합니다. 

함수 안에 할당하는 문장이 있지만 인터프리터가 `b`를 전역 변수로 다루기 원한다면, 다음과 같이 `global` 키워드를 이용해서 선언해야 합니다.

In [15]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9
    
f3(3)

3
6


In [16]:
b

9

In [17]:
f3(3)

3
9


In [18]:
b = 30
b, f3(3)

3
30


(30, None)

## 7.5 클로저

클로저는 내포된 함수 안에서만 의미가 있지만 익명함수와는 다른 개념입니다.

클로저는 __함수 본체에서 정의하지 않고 참조하는 비전역 변수를 포함한 확장 범위를 가진 함수__입니다. 함수가 익명 함수인지 여부는 중요하지 않습니다. __함수 본체 외부에 정의된 비전역 변수에 접근할 수 있다는 것이 중요합니다.__

In [19]:
## ex 7.8
class Average():
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [20]:
avg = Average()
avg(10)

10.0

In [21]:
avg(11)

10.5

In [22]:
avg(12)

11.0

함수는 호출되면 `average()` 함수 객체를 반환합니다. `average()` 함수는 호출될 때마다 받은 인수를 `series` 리스트에 추가하고 현재까지의 평균을 계산해서 출력합니다.

`ex 7.8`을 보면 클래스로 생성된 객체가 데이터를 보관하는 방법은 명확히 알 수 있습니다. `self.series` 객체 속성에 저장되기 때문입니다. 

In [23]:
## ex 7.9
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

In [24]:
avg = make_averager()
avg(10)

10.0

In [25]:
avg(11)

10.5

In [26]:
avg(12)

11.0

하지만 `ex 7.9`을 보면 `series`를 어디서 찾을까요 ?

`make_average()` 함수 본체 안에서 `series = []`로 초기화하고 있으므로 `series`는 이 함수의 지역 변수입니다. `avg(10)`을 호출할 때, `make_averager()`함수는 이미 반환했으므로 지역 범위도 이미 사라진 후입니다. 

그 후 `average` 안에 있는 `series`는 __자유 변수(free variable)__입니다. 자유 변수란 __지역 범위에 바인딩되어 있지 않은 변수를 의미합니다.__ 

![ex7.1](../images/7_1.png)

> __바인딩__
    - 변수에 변수와 관련된 속성을 연관시키는 것

반환된 `averager()`객체를 조사해보면 파이썬이 컴파일된 함수 본체를 나타내는 `__code__`속성 안에 어떻게 지역 변수와 자유 변수의 '이름'을 저장하는지 알 수 있습니다.

In [27]:
avg.__code__.co_varnames

('new_value', 'total')

In [28]:
avg.__code__.co_freevars

('series',)

`series`에 대한 바인딩은 반환된 `avg()`함수의 `__closure__` 속성에 저장됩니다.

In [29]:
avg.__closure__

(<cell at 0x10c4214d0: list object at 0x10c62b5f0>,)

이는 `avg.__code__.co_freevars`의 이름에 대응됩니다.

이 항목은 cell 객체이며 이 객체의 항목은 `cell_contents`속성에서 찾을 수 있습니다.

In [30]:
avg.__closure__[0].cell_contents

[10, 11, 12]

> __cell 객체__
    - 《셀》 객체는 여러 스코프에서 참조하는 변수를 구현하는 데 사용됩니다. 이러한 변수마다, 값을 저장하기 위해 셀 객체가 만들어집니다; 값을 참조하는 각 스택 프레임의 지역 변수에는 해당 변수를 사용하는 외부 스코프의 셀에 대한 참조가 포함됩니다. 값에 액세스하면, 셀 객체 자체 대신 셀에 포함된 값이 사용됩니다. 이러한 셀 객체의 역참조(de-referencing)는 생성된 바이트 코드로부터의 지원이 필요합니다; 액세스 시 자동으로 역참조되지 않습니다. 셀 객체는 다른 곳에 유용하지는 않습니다.

1. __클로저는 함수를 정의할 때 존재하던 자유 변수에 대한 바인딩을 유지하는 함수__


2. __함수를 정의하는 범위가 사라진 후에 함수를 호출해도 자유 변수에 접근 가능__


3. __함수가 '비전역' 외부 변수를 다루는 경우는 그 함수가 다른 함수 안에서 정의된 경우 뿐이라는 점에 주의__

## 7.6 nonlocal 선언

In [31]:
## ex 7.13
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total/count
    return averager

In [32]:
avg = make_averager()

In [33]:
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

`count += 1` 은 `count = count + 1`을 의미하는데 `averager` 본체 안에서 `count` 변수에 할당하고 있기 때문에 `count`를 지역 변수로 만들어서 에러가 발생합니다.

앞선 예제에서는 `series`를 변수에 할당하지 않기 때문에 아무런 문제도 발생하지 않았습니다.

하지만 숫자, 문자열, 튜플 등 불변형은 읽을 수만 있고 값을 갱신할 수 없으므로 재할당을 하면서 `count`라는 지역 변수를 생성하게 되고 자유 변수가 아니게 되므로 클로저에 저장되지 않습니다.

In [34]:
## ex 7.14
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count
    return averager

In [35]:
avg = make_averager()

In [36]:
avg(10)

10.0

In [37]:
avg(11)

10.5

In [38]:
avg(12)

11.0

이를 해결하기 위해선 `nonlocal` 선언을 해주면 됩니다. 변수를 `nonlocal` 선언하면, 함수 안에서 재할당을 하더라도 그 변수는 자유변수로 남아있을 수 있습니다.

새로운 값을 nonlocal 변수에 할당하면 클로저에 저장된 바인딩이 변경됩니다.

In [39]:
avg.__code__.co_freevars

('count', 'total')

## 7.7 간단한 데커레이터 구현하기

In [40]:
! cat ../codes/clockdeco.py

import time

def clock(func):
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed,
                                         name,
                                         arg_str,
                                         result))
        return result
    return clocked


In [41]:
! cat ../codes/clockdeco_demo.py

import time
from clockdeco import clock

@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*'*40, 'Calling snooze(.123)')
    snooze(.123)
    print('*'*40, 'Calling factorial(6)')
    print(' 6! = ', factorial(6))


In [42]:
! python3 ../codes/clockdeco_demo.py

**************************************** Calling snooze(.123)
[0.12443417s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000051s] factorial(1) -> 1
[0.00001036s] factorial(2) -> 2
[0.00001738s] factorial(3) -> 6
[0.00002362s] factorial(4) -> 24
[0.00002982s] factorial(5) -> 120
[0.00003828s] factorial(6) -> 720
 6! =  720


파이썬 인터프리터가 내부적으로 `clocked()`를 `factorial`에 할당했습니다. 

In [43]:
import clockdeco_demo
clockdeco_demo.factorial.__name__

'clocked'

하지만 위 예제는 몇 가지 단점이 존재합니다. 키워드 인수를 지원하지 않고, 데커레이트된 함수의 `__name__`, `__doc__`를 씹습니다. 

`functools.wraps()`데커레이트드를 이용해서 `func`에서 `clocked`로 관련된 속성을 복사합니다. 

In [44]:
! cat ../codes/clockdeco2.py

import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k,w) for k,w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed,
                                         name,
                                         arg_str,
                                         result))
        return result
    return clocked


In [45]:
from clockdeco2 import clock

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

factorial.__name__

'factorial'

## 7.8 표준 라이브러리에서 제공하는 데커레이터

앞서 예제에서 사용한 `functools.wraps()` 데커레이터는 제대로 작동하는 데커레이터를 만들기 위한 헬퍼입니다.

### 7.8.1 `functools.lru_cache()`를 이용한 메모이제이션

`functools.lru_cache()`는 유용한 데커레이터인데, 메모제이션(memozation)을 구현합니다. 

> __메모제이션__
    - 이전에 실행한 값비싼 함수의 결과를 저장함으로써 이전에 사용된 인수에 대해 다시 계산할 필요가 없게 해줍니다. 

__LRU__ 는 __Least Recently Used__의 약자로서, 오랫동안 사용하지 않은 항목을 버림으로써 캐시가 무한정 커지지 않음을 의미합니다. 

In [46]:
## ex 7.18

from clockdeco import clock

@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

print(fibonacci(6))

[0.00000040s] fibonacci(0) -> 0
[0.00000041s] fibonacci(1) -> 1
[0.00006165s] fibonacci(2) -> 1
[0.00000039s] fibonacci(1) -> 1
[0.00000044s] fibonacci(0) -> 0
[0.00000049s] fibonacci(1) -> 1
[0.00005722s] fibonacci(2) -> 1
[0.00011544s] fibonacci(3) -> 2
[0.00044845s] fibonacci(4) -> 3
[0.00000040s] fibonacci(1) -> 1
[0.00000044s] fibonacci(0) -> 0
[0.00000032s] fibonacci(1) -> 1
[0.00003433s] fibonacci(2) -> 1
[0.00008862s] fibonacci(3) -> 2
[0.00000053s] fibonacci(0) -> 0
[0.00000048s] fibonacci(1) -> 1
[0.00019850s] fibonacci(2) -> 1
[0.00000033s] fibonacci(1) -> 1
[0.00000052s] fibonacci(0) -> 0
[0.00000065s] fibonacci(1) -> 1
[0.00019854s] fibonacci(2) -> 1
[0.00023503s] fibonacci(3) -> 2
[0.00047200s] fibonacci(4) -> 3
[0.00069332s] fibonacci(5) -> 5
[0.00123645s] fibonacci(6) -> 8
8


실행 결과는 다음과 같으며 마지막 결과를 제외한 모든 출력은 `clock()` 데커레이터가 생성한 것입니다.

이렇게 구현하면 같은 계산을 여러번 호출하므로 계산 낭비가 심합니다. 하지만 `lru_cache()`를 사용하면 성능이 상당히 개선됩니다. 

In [47]:
## ex 7.19
import functools

from clockdeco import clock

@functools.lru_cache() #1
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

print(fibonacci(6))

[0.00000040s] fibonacci(0) -> 0
[0.00000036s] fibonacci(1) -> 1
[0.00006007s] fibonacci(2) -> 1
[0.00000061s] fibonacci(3) -> 2
[0.00009084s] fibonacci(4) -> 3
[0.00000108s] fibonacci(5) -> 5
[0.00030520s] fibonacci(6) -> 8
8


`lru_cache()` 데커레이터를 일반 함수처럼 호출해야 한다는 점에 주의해야합니다. 이는 __설정 매개변수__를 받기 때문입니다.

결과를 확인해보면 실행 시간이 줄었고 각 n에 대해 연산이 한번만 호출됩니다.

재귀 알고리즘을 쓸만하게 만드는 것 외에 `lru_cache()`는 웹에서 정보를 가져와야 하는 애플리케이션을 만들 때에도 유용합니다.

`lru_cache()`는 두 개의 선택적 인수를 이용해서 설정할 수 있습니다. 

```
functools.lru_cache(maxsize=128, typed=False)
```

- `maxsize` : 얼마나 많은 호출을 저장할지 결정합니다.
> 캐시가 가득차면 가장 오래된 결과를 버리고 공간을 확보합니다. 최적의 성능을 내기 위해 `maxsize`는 __2의 제곱이 되어야 합니다.__


- `typed` : True인 경우 인수의 자료형이 다르면 따로 저장합니다.

`lru_cache`가 결과를 저장하기 위해 딕셔너리를 사용하고, 호출할 때 사용한 위치 인수와 키워드 인수를 키로 사용하므로 데커레이트된 함수가 받는 인수는 모두 __해시 가능__ 해야 합니다.

### 7.8.2 `singledispatch`를 이용한 범용 함수

`functools.singledispatch()` 데커레이터는 각 모듈이 전체 해결책에 기여할 수 있게 해주며, __편집할 수 없는 클래스에 대해서도 특화된 함수를 제공하도록 해줍니다.__ 

일반 함수를 `@singledispatch`로 데커레이트하면, 이 함수는 __범용 함수(generic function)__ 가 됩니다. 즉, 일련의 함수가 첫 번째 인수의 자료형에 따라 서로 다른 방식으로 연산을 수행할 수 있게끔 해줍니다. 

> __generic function__
    - 어떤 하나의 함수가 여러 타입의 인자를 받고, 인자의 타입에 따라 적절한 동작을 하는 함수를 generic function 이라고 합니다.
    

웹 어플리케이션을 디버깅하는 도구를 만들고 있다고 가정해 보겠습니다. 파이썬 객체의 자료형마다 HTML 코드를 생성하고자 합니다.

In [48]:
## ex 7.20

import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

In [49]:
htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [50]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [51]:
htmlize('Heimlich & Co.\n- a game')

'<pre>&#x27;Heimlich &amp; Co.\\n- a game&#x27;</pre>'

In [52]:
htmlize(42)

'<pre>42</pre>'

In [53]:
htmlize(['alpha', 66, {3,2,1}])

'<pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]</pre>'

파이썬에서는 __메서드나 함수의 오버로딩을 지원하지 않으므로__, 서로 다르게 처리하고자 하는 자료형별로 서로 다른 시그너처를 가진 `htmlize()`를 만들 수 없습니다. 

> __함수 시그너처(function signature)__
    - 함수 시그니처란 함수의 원형에 명시되는 매개변수 리스트를 가리킵니다.
    - 만약 두 함수가 매개변수의 개수와 그 타입이 모두 같다면, 이 두 함수의 시그니처는 같다고 할 수 있습니다.
    - 즉, 함수의 오버로딩은 서로 다른 시그니처를 갖는 여러 함수를 같은 이름으로 정의하는 것이라고 할 수 있습니다.

이 때 파이썬에서는 일반적으로 `htmlize()`함수를 디스패치 함수로 변경하고, 일련의 `if/elif/elif` 문을 이용해서 `htmilze_str()`, `htmlize_int()` 등의 특화된 함수를 호출합니다. 

그러면 이 모듈의 사용자가 코드를 확장하기 쉽지 않으며, 다루기도 어렵습니다. 시간이 지나면서 `htmlize()` 디스패치 코드가 커지며, 디스패치 함수와 특화된 함수 간의 결합이 너무 강해집니다. 

In [54]:
def func():
    pass

@singledispatch
def func_single():
    pass

set(dir(func_single)) - set(dir(func))

NameError: name 'singledispatch' is not defined

In [55]:
## ex 7.21

from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return f'<p>{content}</p>'

@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

In [56]:
htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [57]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [58]:
htmlize('Heimlich & Co.\n- a game')

'<p>Heimlich &amp; Co.<br>\n- a game</p>'

In [59]:
htmlize(42)

'<pre>42 (0x2a)</pre>'

In [60]:
print(htmlize(['alpha', 66, {3,2,1}]))

<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>


가능하면 `int`나 `list`와 같은 구상 클래스보다 `numbers.Integral`이나 `abc.MutableSequence`와 같은 추상 클래스를 처리하도록 특화된 함수를 등록하는 것이 좋습니다. 

__추상 클래스로 등록하면 호환되는 자료형을 폭 넓게 지원가능합니다.__ ( _이 개념에 대해서는 11장에서 다시 다룹니다!_ )

`singledispatch` 매커니즘은 특화된 함수를 시스템 어디에나, 어느 모듈에나 등록할 수 있다는 장점이 있습니다. 나중에 새로운 사용자 정의 자료형이 추가된 모듈을 추가할 때도 추가된 자료형을 처리하도록 특화된 함수를 쉽게 추가 가능합니다. 

`singledispatch`는 이보다 더 많은 기능을 제공하므로 공식 문서를 참조하는 것을 추천합니다.
http://www.python.org/dev/peps/pep-0443/

## 7.9 누적된 데커레이터

데커레이터 역시 함수이기에 조합 가능합니다. 

```
@d1
@d2
def f():
```

위처럼 코드를 작성하면 `f = d1(d2(f))` 와 동일합니다.

## 7.10 매개변수화된 데커레이터

소스 코드에 데커레이터를 파싱할 때 파이썬은 데커레이트된 함수를 가져와서 데커레이터 함수의 첫번째 인수로 넘겨줍니다. 그럼 다른 인수들을 어떻게 받는지 예제로 보겠습니다. 

In [61]:
## ex 7.22

registry = []

def register(func):
    print('running register(%s)'% func)
    registry.append(func)
    return func

@register
def f1():
    print('runnning f1')

running register(<function f1 at 0x10c64d3b0>)


In [62]:
print(registry)

[<function f1 at 0x10c64d3b0>]


In [63]:
f1()

runnning f1


### 7.10.1 매개변수화된 등록 데커레이터

In [64]:
## ex 7.23

registry = set()

def register(active=True):
    def decoreate(func):
        print('running register(active=%s)-> decorate(%s)'
              %(active, func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decoreate

@register(active=False)
def f1():
    print('running f1()')
    
@register()
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')

running register(active=False)-> decorate(<function f1 at 0x10c64da70>)
running register(active=True)-> decorate(<function f2 at 0x10c64d950>)


In [65]:
registry

{<function __main__.f2()>}

위 예제를 보면 `decorate`함수가 `active`인수를 받도록 프로그래밍 한 것을 볼 수 있습니다.

데코레이터를 일반함수처럼 사용하려면 다음과 같이 사용해야 합니다.

In [66]:
register()(f3)

running register(active=True)-> decorate(<function f3 at 0x10c64d3b0>)


<function __main__.f3()>

In [67]:
register(active=False)(f2)

running register(active=False)-> decorate(<function f2 at 0x10c64d950>)


<function __main__.f2()>

In [68]:
registry

{<function __main__.f3()>}

여기서 핵심은 `register()`가 `decorate()`를 반환하고, 데커레이트될 함수에 `decorate()`가 적용된다는 것입니다. 

매개변수화된 데커레이터는 일반적으로 데커레이트된 함수를 대체하며, 생성하기 위해 함수를 한 단계 더 내포합니다.

### 7.10.2 매개변수화된 `clock` 데커레이터

앞에서 보였던 `clock` 데커레이터 예제를 이용해서 기능을 추가합니다. 

사용자가 포맷 문자열을 전달해서 데커레이트된 함수가 출력할 문자열을 설정합니다. 

In [69]:
## ex 7.25

import time 

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate

In [70]:
@clock()
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

[0.12805991s] snooze(0.123) -> None
[0.12585939s] snooze(0.123) -> None
[0.12303958s] snooze(0.123) -> None


매개변수화된 데커레이터를 이용할 때 항상 호출 선언`()`을 해줘야 합니다. 

In [71]:
@clock('{name}:{elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze:0.12809375700000025s
snooze:0.1251853660000002s
snooze:0.12803319000000002s


위와 같은 방식으로 적절한 포맷을 커스텀해서 활용 가능합니다. 

> 이 책의 리뷰어들 중 몇몇은 데커레이터 함수를 설명하기 위해 우리가 사용했던 예제와 같은 함수보다는 `__call__()`을 구현하는 클래스로 만들 때 가장 잘 구현된다고 조언했습니다.

## 7.11 요약

1. 결국 우리는 메타프로그래밍의 영역에 들어왔습니다.


2. 내부 함수를 갖지 않는 간단한 데커레이터로 시작해서 두 단계의 내포된 함수를 갖는 매개변수화된 데커레이터까지 구현해봤습니다.


3. 등록 데커레이터는 본질적으로 간단한 메커니즘이지만 고급 파이썬 프레임워크에서 실제 사용되고 있습니다. 


4. 매개변수화된 데커레이터는 거의 최소 두 단계의 내포된 함수를 가지고 있으며, 더 고급 기법을 지원하는 데커레이터를 구현하기 위해 `@functools.wraps`를 사용하는 경우 세 단계 이상 내포되기도 합니다. 


5. 표준 라이브러리 `functools` 모듈에서 제공하는 `@lru_cache()`와 `@singledispatch` 등 함수 데커레이터를 알아보았습니다. 


6. 데커레이터가 실제 작동하는 방식을 이해하려면 __임포트 타임__과 __런타임__의 차이를 알아야 하며, 변수 범위, 클로저, `nonlocal` 선언에 대해서도 자세히 알아야 합니다. (클로저와 `nonlocal`선언을 이해하면 GUI 방식의 이벤트 주도 프로그램이나 콜백을 이용한 비동기 입출력을 구현할 때도 큰 도움이 됩니다.)